# Extract covariates from Google Earth Engine

### Load libraries, intialize GEE connection, define functions

In [124]:
import ee
import eeconvert
import geemap.eefolium as geemap
import pandas as pd
import geopandas as gpd
ee.Initialize()

Function for intersecting points of interest with ee.Image objects and extracting values of bands. 

In [125]:
# df: pandas dataframe with points, coordinates as LONGITUDE, LATITUDE
# id_col: character, column name of ID that will identify points in output file
# batch: character, column name to use for processing in batches
# image: ee.Image name to extract bands from
# outpath: path to write result dataframe

def ee_sample_regions_to_csv(df, id_col, batch, image, outpath):
    # break in batches by years to reduce size of data return
    batches = df[batch].unique()
    # create empty dataframe to append output
    features_df = pd.DataFrame()
    # loop over years
    for i in batches:
        # filter to data from given year
        tmp_df = df[df[batch] == i]
        # convert to geopandas dataframe
        gdf = gpd.GeoDataFrame(tmp_df, geometry = gpd.points_from_xy(tmp_df.LONGITUDE, tmp_df.LATITUDE))
        # convert to ee.FeatureCollection
        fc = eeconvert.gdfToFc(gdf)
        # sample points in batch
        values = image.sampleRegions(**{
            'collection': fc,
            'properties': [id_col],
            'scale': 10})
        # extract data from EE object
        output = values.getInfo()
        # convert to pandas dataframe
        features = output['features']
        for i in range(len(features)):
            item = features[i]
            features_df = features_df.append(item['properties'], ignore_index=True)
    features_df.to_csv(outpath)

### State boundary

In [126]:
ca_fc = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'California'))
ca_geom = ca_fc.geometry()

### FPA-FOD wildfire occurrence data
filter for fires greater than 10 acres (remove classes A and B)

In [127]:
fpa_fod = pd.read_csv('/Users/rebeccawillison/Documents/research/wildfire/wildfires/data/ca_fires.csv',
                        usecols=['FOD_ID', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'STAT_CAUSE_DESCR',
                                 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE'])
fire_classes = ['C', 'D', 'E', 'F', 'G']
wildfires = fpa_fod[fpa_fod['FIRE_SIZE_CLASS'].isin(fire_classes)]

### Static features
USGS National Elevation Dataset 1/3 arc-second (https://developers.google.com/earth-engine/datasets/catalog/USGS_NED?hl=en)      

Global Human Modification Dataset (https://developers.google.com/earth-engine/datasets/catalog/CSP_HM_GlobalHumanModification)

In [94]:
# NED elevation dataset 
NED = ee.Image("USGS/NED").clip(ca_geom)
# calculate slope
slope = ee.Terrain.slope(NED)
# calculate aspect
aspect = ee.Terrain.aspect(NED)
# global human modication dataset
ghm_ic = ee.ImageCollection("CSP/HM/GlobalHumanModification").filterBounds(ca_geom)
ghm = ghm_ic.reduce(ee.Reducer.median()).clip(ca_geom)

# add as bands to create one image
topo = NED.addBands(slope).addBands(aspect).addBands(ghm)

# extract feature values for points
fp = '/Users/rebeccawillison/Documents/research/wildfire/wildfires/data/GEE Layers/StaticFeatures.csv'
ee_sample_regions_to_csv(df = wildfires, id_col = 'FOD_ID', batch = 'FIRE_YEAR', image = topo, outpath = fp)

In [128]:
# LANDFIRE layers
frg = ee.ImageCollection("LANDFIRE/Fire/FRG/v1_2_0").filterBounds(ca_geom).toBands().clip(ca_geom)
mfri = ee.ImageCollection("LANDFIRE/Fire/MFRI/v1_2_0").filterBounds(ca_geom).toBands().clip(ca_geom)
pls = ee.ImageCollection("LANDFIRE/Fire/PLS/v1_2_0").filterBounds(ca_geom).toBands().clip(ca_geom)
pms = ee.ImageCollection("LANDFIRE/Fire/PMS/v1_2_0").filterBounds(ca_geom).toBands().clip(ca_geom)
prs = ee.ImageCollection("LANDFIRE/Fire/PRS/v1_2_0").filterBounds(ca_geom).toBands().clip(ca_geom)
sclass = ee.ImageCollection("LANDFIRE/Fire/SClass/v1_4_0").filterBounds(ca_geom).toBands().clip(ca_geom)
vcc = ee.ImageCollection("LANDFIRE/Fire/VCC/v1_4_0").filterBounds(ca_geom).toBands().clip(ca_geom)
vdep = ee.ImageCollection("LANDFIRE/Fire/VDep/v1_4_0").filterBounds(ca_geom).toBands().clip(ca_geom)
bps = ee.ImageCollection("LANDFIRE/Vegetation/BPS/v1_4_0").filterBounds(ca_geom).toBands().clip(ca_geom)
evc = ee.ImageCollection("LANDFIRE/Vegetation/EVC/v1_4_0").filterBounds(ca_geom).toBands().clip(ca_geom)
evh = ee.ImageCollection("LANDFIRE/Vegetation/EVH/v1_4_0").filterBounds(ca_geom).toBands().clip(ca_geom)
evt = ee.ImageCollection("LANDFIRE/Vegetation/EVT/v1_4_0").filterBounds(ca_geom).toBands().clip(ca_geom)
esp = ee.Image("LANDFIRE/Vegetation/ESP/v1_2_0/CONUS").clip(ca_geom) 
gap = ee.Image("USGS/GAP/CONUS/2011").clip(ca_geom)

lf = frg.addBands(mfri).addBands(pls).addBands(pms).addBands(prs).addBands(sclass).addBands(vcc).addBands(vdep) \
        .addBands(bps).addBands(evc).addBands(evh).addBands(evt).addBands(esp).addBands(gap)

# extract feature values for points
fp = '/Users/rebeccawillison/Documents/research/wildfire/wildfires/data/GEE Layers/LANDFIREFeatures.csv'
ee_sample_regions_to_csv(df = wildfires, id_col = 'FOD_ID', batch = 'FIRE_YEAR', image = lf, outpath = fp)

### Low temporal frequency layers
Population density (https://developers.google.com/earth-engine/datasets/catalog/CIESIN_GPWv411_GPW_Population_Density)

MODIS yearly land cover (https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1)

Cropland data layer (https://developers.google.com/earth-engine/datasets/catalog/USDA_NASS_CDL)

In [118]:
# population density, every 5 years
pop = ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Density").filterBounds(ca_geom).toBands().clip(ca_geom)
# extract feature values for points
fp = '/Users/rebeccawillison/Documents/research/wildfire/wildfires/data/GEE Layers/PopDensFeatures.csv'
ee_sample_regions_to_csv(df = wildfires, id_col = 'FOD_ID', batch = 'FIRE_YEAR', image = pop, outpath = fp)

In [113]:
# MODIS land cover classification, yearly
modis = ee.ImageCollection("MODIS/006/MCD12Q1").filterBounds(ca_geom).toBands().clip(ca_geom)
# extract feature values for points
fp = '/Users/rebeccawillison/Documents/research/wildfire/wildfires/data/GEE Layers/MODISFeatures.csv'
ee_sample_regions_to_csv(df = wildfires, id_col = 'FOD_ID', batch = 'FIRE_YEAR', image = modis, outpath = fp)

In [123]:
# USDA NASS CDL classification, yearly
def getCrop(image):
    return(image.select('cropland'))
band_nos = list(range(3, 15))
cdl = ee.ImageCollection("USDA/NASS/CDL").filterBounds(ca_geom).map(getCrop).toBands().clip(ca_geom).select(band_nos)
# extract feature values for points
fp = '/Users/rebeccawillison/Documents/research/wildfire/wildfires/data/GEE Layers/CDLFeatures.csv'
ee_sample_regions_to_csv(df = wildfires, id_col = 'FOD_ID', batch = 'FIRE_YEAR', image = cdl, outpath = fp)

### High temporal freqency layers
MODIS Fire_cci Burned Area Pixel product (https://developers.google.com/earth-engine/datasets/catalog/ESA_CCI_FireCCI_5_1)
GRIDMET Gridded Surface Meteorological Dataset (https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_GRIDMET)
GRIDMET Drought Dataset (https://developers.google.com/earth-engine/datasets/catalog/GRIDMET_DROUGHT)

In [129]:
# MODIS layers - download individual bands to reduce data size due to monthly frequency
# observed flag
def getFlag(image):
    return(image.select('ObservedFlag'))
flag = ee.ImageCollection("ESA/CCI/FireCCI/5_1").filterBounds(ca_geom).map(getFlag).toBands().clip(ca_geom)
fp = '/Users/rebeccawillison/Documents/research/wildfire/wildfires/data/GEE Layers/MODISFlagFeatures.csv'
ee_sample_regions_to_csv(df = wildfires, id_col = 'FOD_ID', batch = 'FIRE_YEAR', image = flag, outpath = fp)

In [130]:
# burned area
def getBurned(year):
    start_date = ee.Date(year + '-01-01')
    end_date = ee.Date(year + '-01-01')
    cci = ee.ImageCollection('ESA/CCI/FireCCI/5_1').filterBounds(ca_geom).filterDate(start_date, end_date) \
            .select('BurnDate').max()
    

fp = '/Users/rebeccawillison/Documents/research/wildfire/wildfires/data/GEE Layers/MODISCCIFeatures.csv'
ee_sample_regions_to_csv(df = wildfires, id_col = 'FOD_ID', batch = 'FIRE_YEAR', image = cci, outpath = fp)



'2015-01-01'